Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a [kaggle competition](https://www.kaggle.com/c/whiskey-201911/) We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

We are going to run increasingly sophisticated classification models on our whisky reviews in parts 1, 2, and 3. For each of parts 1, 2, and 3, submit your best model's results to the Kaggle competition to measure `generalization accuracy` -- i.e. how well the model performs on new data.

##1. Classifier based on TfIdf vectorization of reviews

### Follow Along 

1. Join the Kaggle Competition
2. Download the data
3. Train and hyperparameter tune a model using an sklearn pipeline

### 1.0 Setup

#### 1.0.1 Get spacy and restart runtime

In [ ]:
#YOUR CODE HERE
%%time

!python -m spacy download en_core_web_md


2022-12-06 21:50:45.974100: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 2.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
CPU times: user 224 ms, sys: 43 ms, total: 267 ms
Wall time: 14 s


#### 1.0.2 import necessary packages, load spacy

In [ ]:
import pandas as pd
import re

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
import spacy

Load `spacy`

In [ ]:
nlp = spacy.load("en_core_web_md")

#### 1.0.3 Load Kaggle Whisky Competition Data
The goal is to predict the rating from the review text

In [ ]:
# !!!!! You may need to change the path !!!!!
# You can download these datasets from the Kaggle in-class 
# competition for your cohort.
# from google.colab import files
# uploaded = files.upload()
 
train = pd.read_csv('train.csv', usecols=["description", 'category'])
test = pd.read_csv('test.csv', usecols=['id','description'])

In [ ]:
train

,description,category
0,A marriage of 13 and 18 year old bourbons. A m...,2
1,There have been some legendary Bowmores from t...,1
2,This bottling celebrates master distiller Park...,2
3,What impresses me most is how this whisky evol...,1
4,"A caramel-laden fruit bouquet, followed by une...",2
...,...,...
2581,"Earthy, fleshy notes with brooding grape notes...",1
2582,With its overt floral perfume notes and the sc...,4
2583,"An unaged whiskey from Carroll County, Iowa, w...",3
2584,"Fiery peat kiln smoke, tar, and ripe barley on...",1


### 1.1 Clean Text

In [ ]:
def clean_doc(text):
  # COMPLETE THE CODE IN THIS CELL
  # remove new line characters
  text = text.replace('\\n', ' ')
  # remove numbers from the text
  text = re.sub(r"[09]","", text)
  # remove multiple white spaces
  text = re.sub(r"[\s+]", " ", text)

  # case normalize and strip extra white spaces on the far left and right hand side
  #text = text.lower()
  text = text.strip()
  return text

train['description'] = train['description'].apply(clean_doc)
test['description'] = test['description'].apply(clean_doc)


In [ ]:
test

,id,description
0,955,"Think carnival aromas—the good ones, anyway—me..."
1,3532,"A blend of three bourbons, between 6 and 12 ye..."
2,1390,"The nose is focused on cereal, hints of fresh ..."
3,1024,Swiss-based Chapter 7 released this 1 year old...
4,1902,Valkyrie replaces the current Dark Origins exp...
...,...,...
283,1834,"Aged in a sherry cask, and the sherry is clean..."
284,2848,There have been some legendary Hart Brothers b...
285,3874,"Aromas hint at a bitterly astringent, almost b..."
286,2377,"Think corn: roasted, canned, creamed, mashed, ..."


### 1.2 Split training data into Feature Matrix `X` and Target Vector `y`

In [ ]:
import numpy as np
target = 'category'
# COMPLETE THE CODE IN THIS CELL
y = train[target]
X = train.drop(columns=target)

y = pd.DataFrame(y)
y

,category
0,2
1,1
2,2
3,1
4,2
...,...
2581,1
2582,4
2583,3
2584,1


### 1.3 Specify the Model and Define the Pipeline Components

For the classifier model, you can try any or several of 
* `RandomForestClassifier()` or `GradientBoostingClassifier()` from the `sklearn` library
* `XGBClassifier()` from the `xgboost` library
* `CatboostClassifier()` from the `catboost` library
* `LGBMClassifier()` from the `lightgbm` library


In [ ]:
from pandas.core.computation.expr import tokenize_string
# limit max_features to 500 to speed up training on Colab.
# COMPLETE THE CODE IN THIS CELL
vect = TfidfVectorizer(stop_words='english', min_df=0.25)
tf_idf = vect.fit_transform(train.description)
tf_dtm = pd.DataFrame(tf_idf.todense(), columns=[vect.get_feature_names_out()])
clf = RandomForestClassifier(n_jobs=100)

clf.fit(tf_dtm, y)
#pipe = Pipeline([('clf',  clf)])


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
<ipython-input-14-60641e501748>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(tf_dtm, y)


RandomForestClassifier(n_jobs=100)

### 1.4 Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
# COMPLETE THE CODE IN THIS CELL
# Parameters to search in dictionary 
#parameters = {
    #'clf__max_depth':(100, 150)}

# Implement a grid search with cross-validation
# grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
# grid_search.fit(X,y)

# Display the best score from the grid search
score = clf.score(tf_dtm, y)
score

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


0.8298530549110595

In [ ]:
# Display the best parameters from the grid search
print(grid_search.best_params_)

NameError: ignored

### 1.5 Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so only submit when your predicted test accuracy is the highest you can make it. For this competition the max daily submissions are capped at **20**.  The submission file is made from the results of running your best model on the **test data set**, for which we don't get the targets.<br><br>

In [ ]:
# COMPLETE THE CODE IN THIS CELL
# Predictions on **test** sample
vect1 = TfidfVectorizer(stop_words='english', min_df=0.25)
tf_idf1 = vect1.fit_transform(test.description)
tf_dtm1 = pd.DataFrame(tf_idf1.todense(), columns=[vect.get_feature_names_out()])
tf_dtm1
pred = clf.predict(tf_dtm1)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
# COMPLETE THE CODE IN THIS CELL
sub = pd.Series(pred)
submission = pd.DataFrame(test.id, columns=['id'])
submission['category'] = sub
submission['category'] = submission['category'].astype('int64')


In [ ]:
# Make Sure the Category is an Integer
submission


,id,category
0,955,2
1,3532,3
2,1390,1
3,1024,1
4,1902,1
...,...,...
283,1834,1
284,2848,1
285,3874,2
286,2377,1


In [ ]:
# # Save your Submission File
# # Best to Use an Integer or Timestamp for different versions of your model
# submission_number = 0

submission.to_csv('submission.csv', index=False)
# submission_number += 1

In [ ]:
# Download submission to local machine from this Google Colab notebook
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 1.6 Submit your results to `kaggle` and get your score

First, upload the `kaggle.json` API token file from your local machine.<br>
Do this by clicking the file icon in the left sidebar, <br>
then clicking file icon with an up arrow inside it at the upper left, <br>
then navigating to and selecting the `kaggle.json` file in your local machine.<br>
`kaggle.json` is usually found in a folder called `.kaggle` on your local machine, <br>
if you have a Kaggle account. Note that there is no cost involved in <br>registering for a Kaggle account.<br><br>

Then: make a folder `/root/.kaggle` in this notebook,<br>
and copy your `kaggle.json` file into the `/root/.kaggle/` folder

In [ ]:
!mkdir /root/.kaggle/
!mv /kaggle.json /root/.kaggle/ 
!chmod 600 /root/.kaggle/kaggle.json # to safeguard your privacy
!ls -l /root/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists
mv: cannot stat '/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
total 0


Now you can submit your predictions to Kaggle
directly from this Colab notebook!<br>


In [ ]:
!kaggle competitions submit whiskey-201911 -f submission1.csv -m "submission"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


You can check your score on the [Kaggle Whisky Competition website](https://www.kaggle.com/c/whiskey-201911/submissions), 
<br>on the "My Submissions" tab:

_This submission got a score of $0.94186$_

## Challenge

You're trying to achieve a minimum of 75% Accuracy on your model.

## 2. Add Latent Semantic Indexing to your pipeline (Learn)
<a id="p2"></a>

### Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (LSI) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### 2.1 Define Pipeline Components

Nest pipelines to perform SVD on our vectorization (LSA)

In [ ]:
# COMPLETE THE CODE IN THIS CELL
# Transforming our Vectorization with SVD is how LSA generates topic columns
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=2)
svd = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, )

# vectorizer and classifier like before
clf = XGBClassifier()

# LSA pipeline with vectorizer & truncated SVD
lsa = Pipeline([('vect', vect), ('svd', svd)])

# combine LSA pipeline together with classifier
pipe = Pipeline([('lsa', lsa), ('clf', clf)])

### 2.2 Define Your grid search space and run a grid search with cross-validation
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
# COMPLETE THE CODE IN THIS CELL
parameters = {
    'lsa__svd__n_components': [100],
    'lsa__vect__max_df': [.20, .25,],
    'clf__max_depth': [6,8,10]
}

grid_search = GridSearchCV(pipe,parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X.description, y.category )

Fitting 3 folds for each of 6 candidates, totalling 18 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('lsa',
                                        Pipeline(steps=[('vect',
                                                         TfidfVectorizer(min_df=2,
                                                                         ngram_range=(1,
                                                                                      2),
                                                                         stop_words='english')),
                                                        ('svd',
                                                         TruncatedSVD(n_components=10,
                                                                      n_iter=10))])),
                                       ('clf', XGBClassifier())]),
             n_jobs=-1,
             param_grid={'clf__max_depth': [6, 8, 10],
                         'lsa__svd__n_components': [100],
                         'lsa__vect__max_df': [0.2, 0.25]},
         

In [ ]:
grid_search.best_score_

0.8990719257540604

In [ ]:
grid_search.best_params_

{'clf__max_depth': 6, 'lsa__svd__n_components': 100, 'lsa__vect__max_df': 0.25}

### 2.3 Make a Submission File
See section $1.6$ above for instructions on how to submit your results file to `kaggle` and get your score

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,4
3,1024,1
4,1902,1


In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'submission.csv', index=False)
#submission_number +=1

In [ ]:
# Download submission to your local machine from this Colab notebook
from google.colab import files
files.download(f'submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# 3. Add Spacy Word Embeddings
<a id="p3"></a>

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use document vectors made from those word embeddings as your features for a classification model.
4. Make a submission to Kaggle 

### 3.1 Process the data set with spacy

In [ ]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here
nlp = spacy.load("en_core_web_md")

def get_word_vectors(docs):
    # YOUR CODE HERE
    return [nlp(doc).vector for doc in docs]

X_train_emb = get_word_vectors(train['description'])
X_test_emb = get_word_vectors(test['description'])

In [ ]:
rfc = RandomForestClassifier(oob_score=True)

rfc.fit(X_train_emb, y)

<ipython-input-40-1eaa4c79f3ca>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train_emb, y)


RandomForestClassifier(oob_score=True)

In [ ]:
# massively overfit with the Random Forest
print('Training Accuracy: ', rfc.score(X_train_emb, y))

Training Accuracy:  1.0


Here we use oob_score_ (out-of-bag score) as a **proxy** for the test score;<br>
for your submission, you will predict on the test set, as before

In [ ]:
# validation looks decent without any tuning

rfc.oob_score_

0.6790409899458624

### 3.2 Make a Submission File
See section $1.6$ above for instructions on how to submit your results file to `kaggle` and get your score

### Make a Submission File

In [ ]:
# YOUR CODE HERE

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
#submission_number = 2
submission.to_csv(f'submission_rfc.csv', index=False)

In [ ]:
# Download submission to local machine from Google Colab
from google.colab import files
files.download(f'submission_rfc.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 3.3 Submit your predictions to Kaggle


---



In [ ]:
# YOUR CODE HERE 

# Post Lecture Assignment (Stretch)
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. <br>
Once you've accomplished that, do (1), and either (2) or (3): 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do people create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?

2. Singular Value Decomposition (SVD) is one of the most important and powerful methods in Applied Mathematics and in all of Machine Learning.  Principal Components Analysis (PCA) -- which we used in Module 2 -- is closely releated to SVD. Research SVD using the resources below. Then write a few paragraphs explaining -- in your own words -- your understanding of SVD and why it has become so important in Machine Learning. As you write, pretend that you will be presenting this summary orally as an answer to a question during a job interview.<br>

* [Daniela Witten](https://www.danielawitten.com/), a Professor of Mathematical Statistics at the University of Washington, recently penned a highly amusing and informative [tweetstorm](https://twitter.com/WomenInStat/status/1285611042446413824) about SVD, well worth reading!<br>
* [Stanford University Lecture on SVD](https://www.youtube.com/watch?v=P5mlg91as1c) <br>
* [StatQuest Principal Components Analysis](https://www.youtube.com/watch?v=FgakZw6K1QQ)<br>
* [Luis Serrano Principal Components Analysis](https://www.youtube.com/watch?v=g-Hb26agBFg)<br>

3. Research which other models can be used for text classification -- see [Multi-Class Text Classification Model Comparison and Selection](https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568)
  - Try a few other classical machine learning models, and compare with the gradient boosting results 
  - Neural Networks are becoming more popular for document classification. Why is that the case? 
  - If you have the time and interest, check out this [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google
   